In [62]:
import time
import re
from typing import Any
from specparser.amt import loader, schedules, table, tickers, strings

amt_path = "../data/amt.yml"
prices_path = "../data/prices.parquet"
chain_path = "../data/futs.csv"
override_path = "../data/overrides.csv"

year = 2025
month = 1
ntrc = "N"

_MEMOIZE_ENABLED = True

_ASSET_STRADDLE_TICKER_CACHE = {}
_ASSET_STRADDLE_TICKER_COLUMNS=["name","ticker","field"]

def split_ticker(ticker,year,month):
        if ":" in ticker:
            ticker_pre,ym,ticker_post =  ticker.split(":",2)
            if ym<f"{year}-{month:02d}":
                return ticker_pre
            else:
                return ticker_post
        else:
            return ticker

def make_fut_ticker(fut_code,fut_month_map,min_year_offset,market_code,qualifier):
    month_map = "FGHJKMNQUVXZ"
    fut_month_code = fut_month_map[month - 1]
    opt_month_code = month_map[month - 1]
    year_offset = max(1 if fut_month_code < opt_month_code else 0, min_year_offset)
    fut_year = year + year_offset
    return  f"{fut_code}{qualifier}{fut_month_code}{fut_year} {market_code}"

def asset_straddle_ticker_key(
    asset: str, 
    strym: str, 
    ntrc:str,
    vol:dict,
    hedge:dict
):
    if vol["Source"]=="BBG_LMEVOL":
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="fut":
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="nonfut" and ":" in hedge["Ticker"]:
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="cds" and ":" in hedge["hedge"]:
        cache_key = asset+"|"+strym+"|"+ntrc
    elif hedge["Source"]=="cds" and ":" in hedge["hedge1"]:
        cache_key = asset+"|"+strym+"|"+ntrc
    else: 
        cache_key = asset
    return cache_key

def get_asset_straddle_tickers(asset: str, strym: str, ntrc:str, amt_path: str) -> dict[str,Any]:

    ticker_dict = dict(
        orientation="row",
        columns=_ASSET_STRADDLE_TICKER_COLUMNS,
        rows = []
    )

    asset_data = loader.get_asset(amt_path,asset)
    if asset_data is None : return ticker_dict
    hedge = asset_data["Hedge"]
    if hedge is None: return ticker_dict
    vol = asset_data["Vol"]
    if vol is None: return ticker_dict

    if ntrc=="F":
        if vol.get("Far","")=="": return ticker_dict
        if vol.get("Near","")==vol.get("Far",""): return ticker_dict
        if vol.get("Far","")=="NONE": return ticker_dict

    cache_key = asset_straddle_ticker_key(asset,strym,ntrc,vol,hedge)
    print(f"cache_key:{cache_key}")

    if _MEMOIZE_ENABLED and cache_key in _ASSET_STRADDLE_TICKER_CACHE:
        return _ASSET_STRADDLE_TICKER_CACHE[cache_key]

    year, month = int(strym[0:4]), int(strym[5:7])

    if vol["Source"]=="BBG" and ntrc=="N":
       ticker_dict["rows"].append(["vol",vol["Ticker"],vol["Near"]])

    if vol["Source"]=="BBG" and ntrc=="F":
        ticker_dict["rows"].append(["vol",vol["Ticker"],vol["Far"]])

    if vol["Source"]=="CV":
        ticker_dict["rows"].append(["vol",vol["Near"],"none"])

    if vol["Source"]=="BBG_LMEVOL":
        vol_ticker =  make_fut_ticker(
            fut_code=hedge["fut_code"],
            fut_month_map=hedge["fut_month_map"],
            min_year_offset=hedge["min_year_offset"],
            market_code=hedge["market_code"],
            qualifier="R"
        )
        ticker_dict["rows"].append(["vol",vol_ticker,"PX_LAST"])

    if hedge["Source"]=="fut":
        fut_ticker = make_fut_ticker(
            fut_code=hedge["fut_code"],
            fut_month_map=hedge["fut_month_map"],
            min_year_offset=hedge["min_year_offset"],
            market_code=hedge["market_code"],
            qualifier=""
        )
        ticker_dict["rows"].append(["hedge",fut_ticker,"PX_LAST"])

    if hedge["Source"]=="nonfut":
        ticker_dict["rows"].append(["hedge",split_ticker(hedge["Ticker"],year,month),hedge["Field"]])

    if hedge["Source"]=="cds":
        ticker_dict["rows"].append(["hedge",split_ticker(hedge["hedge"],year,month),"PX_LAST"])
        ticker_dict["rows"].append(["hedge1",split_ticker(hedge["hedge1"],year,month),"PX_LAST"])

    if hedge["Source"]=="calc":
        ticker_dict["rows"].append(["hedge1",f"{hedge['ccy']}_fsw0m_{hedge['tenor']}",""])
        ticker_dict["rows"].append(["hedge2",f"{hedge['ccy']}_fsw6m_{hedge['tenor']}",""])
        ticker_dict["rows"].append(["hedge3",f"{hedge['ccy']}_pva0m_{hedge['tenor']}",""])
        ticker_dict["rows"].append(["hedge4",f"{hedge['ccy']}_pva6m_{hedge['tenor']}",""])
    
    if _MEMOIZE_ENABLED:
        _ASSET_STRADDLE_TICKER_CACHE[cache_key] = ticker_dict

    return ticker_dict


In [ ]:
#asset_tickers = get_asset_straddle_tickers("LA Comdty TKRZ","2025-01","N",amt_path)
asset_tickers = table.table_bind_rows(
    get_asset_straddle_tickers("LA Comdty","2025-01","N",amt_path),
    get_asset_straddle_tickers("LA Comdty","2025-01","F",amt_path),
    get_asset_straddle_tickers("XB Comdty","2001-01","N",amt_path),
    get_asset_straddle_tickers("SI Comdty","2001-01","F",amt_path),
    get_asset_straddle_tickers("SB Comdty","2001-01","N",amt_path),
    get_asset_straddle_tickers("S Comdty","2001-01","F",amt_path),
    get_asset_straddle_tickers("IBOXUMAE Curncy","2025-01","N",amt_path),
    get_asset_straddle_tickers("IBOXUMAE Curncy","2025-02","N",amt_path),
    get_asset_straddle_tickers("IBOXUMAE Curncy","2025-03","N",amt_path),
    get_asset_straddle_tickers("EURUSD Curncy","2025-01","N",amt_path),
    get_asset_straddle_tickers("USSWAP10 Curncy","2025-01","N",amt_path)
)
print("-"*40)
print(_ASSET_STRADDLE_TICKER_CACHE.keys())
table.show_table(asset_tickers)

cache_key:IBOXUMAE Curncy|2025-01|N
cache_key:IBOXUMAE Curncy|2025-02|N
cache_key:IBOXUMAE Curncy|2025-03|N
cache_key:EURUSD Curncy
cache_key:USSWAP10 Curncy
----------------------------------------
dict_keys(['IBOXUMAE Curncy|2025-01|N', 'EURUSD Curncy', 'USSWAP10 Curncy', 'IBOXUMAE Curncy|2025-02|N', 'IBOXUMAE Curncy|2025-03|N'])


,name,ticker,field
0,vol,CREDIT.CVOL.CDX_NA_IG.1M.CVOL_ATM.CVOL_VOL.CVO...,none
1,hedge,IBOXUMAE CBBT Curncy,PX_LAST
2,hedge1,USSWAP5 CMPN Curncy,PX_LAST
3,vol,CREDIT.CVOL.CDX_NA_IG.1M.CVOL_ATM.CVOL_VOL.CVO...,none
4,hedge,IBOXUMAE CBBT Curncy,PX_LAST
5,hedge1,USSWAP5 CMPN Curncy,PX_LAST
6,vol,CREDIT.CVOL.CDX_NA_IG.1M.CVOL_ATM.CVOL_VOL.CVO...,none
7,hedge,IBOXUMAE CBBT Curncy,PX_LAST
8,hedge1,USSWAP5 CMPN Curncy,PX_LAST
9,vol,EURUSDV1M CMPN Curncy,PX_LAST
